# TVB-NEST: Bridging multiscale activity by co-simulation

## Step-by-step learn how to perform a co-simulation embedding spiking neural networks into large-scale brain networks using TVB.

In [ ]:
from IPython.core.display import Image, display
display(Image(filename='./pics/ConceptGraph1.png',  width=1000, unconfined=False))

In [ ]:
display(Image(filename='./pics/ConceptGraph2.png',  width=1000, unconfined=False))

## tvb-multiscale toolbox:

### https://github.com/the-virtual-brain/tvb-multiscale

For questions use the git issue tracker, or write an e-mail to me: dionysios.perdikis@charite.de

# TVB - NEST co-simulation 

## Linear TVB mean field model

For every region node $n\prime$ modelled as a mean-field node in TVB:

(Post)Synaptic gating dynamics (i.e., proportion of synapse channels open at any given time):

$\tau\dot{R_{n\prime}} = - {R_{n\prime}}(t) + G\sum_{{m}\neq {n\prime}}C_{{m}{n\prime}}R_{m}(t-\tau_{{m}{n\prime}})(t) + I_o(t) $

for $n\prime$ regions modelled in TVB ($m$ all nodes), 

and

$\tau_{Rin}\dot{R_{n}} = - {R_{n}}(t) + {Rin_{n}}(t) $

for $n$ region modelled in NEST, and updating TVB via the ${Rin_{n}}(t)$ instant rate.

 ${Rin_{n\prime}}(t) = 0.0$ for all $n\prime$ TVB nodes.

### Parameters:

- structural TVB connectivity weights $C_{{m\prime}{n\prime}}$ (${m\prime}->{n\prime}$)
- structural TVB connectivity delays $\tau_{{m\prime}{n\prime}}$  (${m\prime}->{n\prime}$)
- global structural brain connectivity coupling constant $G$
- overall effective external input current $I_o = 0.0 except for stimulua application where I_o = 100$ 
- time constant $\tau = 10 ms$ 
- time constant of linear integration of NEST instant rate update $\tau_{Rin} = 10 ms$ 


## Cerebellum Spiking network model in NEST

To be filled in!...

## TVB to NEST coupling
TVB couples to NEST via instantaneous spike rate $ interface_{weight} * R(t) $, 

Spike generator NEST devices are used as TVB "proxy" nodes and generate spike trains 

$ \left[ \sum_k \delta(t-\tau_{n\prime n}-{t_j}^k) \right]_{j \in n\prime} $



In [ ]:
display(Image(filename='./pics/Rate_cereb_linear.png',  width=1000, unconfined=False))

## NEST to TVB update

A NEST spike detector device is used to count spike for each time step, and convert it to an instantaneous population mean rate that overrides an auxiliary TVB state variables $R_{in}(t)$:

$ {R_{in_{n}}}(t) =  \frac{\sum_j\left[ \sum_k \delta(t-\tau_n-{t_j}^k) \right]_{j \in R_n}}{nNeurons * dt} $ in  spikes/sec.

This update process concerns only the TVB region nodes that are simulated exclusively in NEST, as spiking networks. All the rest of TVB nodes will follow the equations of the mean field model described above.


In [ ]:
display(Image(filename='./pics/NESTtoTVB_cereb_linear.png',  width=1000, unconfined=False))

## Simulator loop

### Simulating several (i.e., 4) NEST time steps for every 1 TVB time step for stable integration

# WORKFLOW:

In [ ]:
from collections import OrderedDict
import time
import numpy as np

from tvb.basic.profile import TvbProfile
TvbProfile.set_profile(TvbProfile.LIBRARY_PROFILE)

from tvb_multiscale.tvb_nest.config import *
home_path = "/home/docker/packages/tvb-multiscale"
notebooks_path = os.path.join(home_path, "docs/documented_example_notebook")
config = Config(output_base=os.path.join(notebooks_path, "outputs_Cereb"))
config.figures.SHOW_FLAG = True 
config.figures.SAVE_FLAG = True
config.figures.FIG_FORMAT = 'png'
config.figures.DEFAULT_SIZE= config.figures.NOTEBOOK_SIZE
FIGSIZE = config.figures.DEFAULT_SIZE

from tvb_multiscale.core.plot.plotter import Plotter
plotter = Plotter(config.figures)

# For interactive plotting:
# %matplotlib notebook  

# Otherwise:
%matplotlib inline 

## 1. Load structural data <br> (minimally a TVB connectivity)  <br> & prepare TVB simulator  <br> (region mean field model, integrator, monitors etc)

In [ ]:
from tvb.simulator.models.linear_with_stimulus import Linear
from tvb.datatypes.connectivity import Connectivity
from tvb.simulator.cosimulator import CoSimulator
from tvb.simulator.integrators import HeunStochastic
from tvb.simulator.monitors import Raw  # , Bold, EEG


# Create a TVB simulator and set all desired inputs
# (connectivity, model, surface, stimuli etc)
# We choose all defaults in this example
simulator = CoSimulator()
model_params = {"I_o": np.array([0.0]), "G": np.array([16.0]), "tau": np.array([10.0]), "tau_rin": np.array([10.0])}
simulator.model = Linear(**model_params)

simulator.integrator = HeunStochastic()
simulator.integrator.dt = 0.1
simulator.integrator.noise.nsig = np.array([0.001])


# Load connectivity
# config.DEFAULT_CONNECTIVITY_ZIP = "/home/docker/packages/tvb_data/tvb_data/mouse/allen_2mm/ConnectivityAllen2mm.zip"                                  
# connectivity = Connectivity.from_file(config.DEFAULT_CONNECTIVITY_ZIP)
import os
DATA_PATH = os.path.join(home_path, "examples/data")
w = np.loadtxt(os.path.join(DATA_PATH, "mouse_cereb_sum_weights.txt"))
# t = np.loadtxt(os.path.join(DATA_PATH, "tract_lengths_Count_plusCRBL.txt"))
# forcing one time step delay for all connections:
speed = 4.0
t = speed * simulator.integrator.dt * np.ones(w.shape)  
# brain_regions_path = os.path.join(DATA_PATH, "centres_brain_MNI.txt")
# rl = np.loadtxt(brain_regions_path,dtype="str", usecols=(0,))
with open(os.path.join(DATA_PATH, "mouse_cereb_regions_labels.txt"), "r") as text: 
    rl = [] 
    for line in text: 
        rl.append(line)
rl = np.array(rl)
# c = np.loadtxt(brain_regions_path, usecols=range(1,3))
c = np.random.uniform((w.shape[0], 3))
connectivity=Connectivity(region_labels=rl, weights=w, centres=c, tract_lengths=t)

# Normalize connectivity weights
connectivity.weights = connectivity.scaled_weights(mode="region")
connectivity.weights /= np.percentile(connectivity.weights, 99)
connectivity.weights[connectivity.weights > 1.0] = 1.0

connectivity.speed = np.array([speed])
connectivity.configure()

#white_matter_coupling = coupling.Linear(a=0.014)
simulator.connectivity = connectivity

simulator.model.I_o = simulator.model.I_o[0] * np.ones((simulator.connectivity.number_of_regions, ))

simulator.initial_conditions = np.zeros((2, 2, simulator.connectivity.number_of_regions, 1))

mon_raw = Raw(period=1.0)  # ms
simulator.monitors = (mon_raw, )

plotter.plot_tvb_connectivity(simulator.connectivity);

## 2. Build and connect the NEST network model <br> (networks of spiking neural populations for fine-scale <br>regions, stimulation devices, spike detectors etc)

In [ ]:
# Select the regions for the fine scale modeling with NEST spiking networks
nest_nodes_ids = []
for i_region, reg_lbl in enumerate(simulator.connectivity.region_labels):
    if "cereb" in reg_lbl.lower():
        nest_nodes_ids.append(i_region)  # the indices of fine scale regions modeled with NEST
if len(nest_nodes_ids) == 0:
    nest_nodes_ids = [simulator.connectivity.number_of_regions-1]

print(["%d. %s" % (nest_node_id, simulator.connectivity.region_labels[nest_node_id]) for nest_node_id in nest_nodes_ids])


In [ ]:
connections_to_cereb = simulator.connectivity.weights[:, nest_nodes_ids[0]]
sorted_connections_to_cereb = np.argsort(connections_to_cereb)[::-1]
print("sorted_connections_to_cereb =\n") 
for conn_id in sorted_connections_to_cereb:
    print("\n%d. %s, w = %g" % 
          (conn_id, simulator.connectivity.region_labels[conn_id], connections_to_cereb[conn_id]))
connections_from_cereb = simulator.connectivity.weights[nest_nodes_ids[0], :]
sorted_connections_from_cereb = np.argsort(connections_from_cereb)[::-1]
print("sorted_connections_from_cereb =\n") 
for conn_id in sorted_connections_from_cereb:
    print("\n%d. %s, w = %g" % 
          (conn_id, simulator.connectivity.region_labels[conn_id], connections_from_cereb[conn_id]))

In [ ]:
stim_node_id = 42
print("TVB stimulus to %d. Right Intermediate reticular nucleus, w = %g" % 
      (stim_node_id, connections_to_cereb[stim_node_id]))
connections_from_stim = simulator.connectivity.weights[stim_node_id, :]
sorted_connections_from_stim = np.argsort(connections_from_stim)[::-1]
print("which connects to...")
print("sorted_connections_from 42. Right Intermediate reticular nucleus =\n") 
for conn_id in sorted_connections_from_stim:
    print("\n%d. %s, w = %g" % 
          (conn_id, simulator.connectivity.region_labels[conn_id], connections_from_stim[conn_id]))


In [ ]:
# Build a NEST network model with the corresponding builder
from tvb_multiscale.tvb_nest.nest_models.builders.models.cereb import CerebBuilder

# Using all default parameters for this example
nest_model_builder = CerebBuilder(simulator, nest_nodes_ids, 
                                  os.path.join(DATA_PATH, "cerebellar_cortex_scaffold_dcn.hdf5"),
                                  config=config)
nest_model_builder.modules_to_install = ["cereb"]

# or...

# # ----------------------------------------------------------------------------------------------------------------
# # ----Uncomment below to modify the builder by changing the default  builder configuration------------------------
# # ----------------------------------------------------------------------------------------------------------------
# import h5py

# # Synapse parameters: in E-GLIF, 3 synaptic receptors are present:
# # the first is always associated to exc,
# # the second to inh,
# # the third to remaining synapse type
# Erev_exc = 0.0  # [mV]	# [Cavallari et al, 2014]
# Erev_inh = -80.0  # [mV]
# # tau_exc for pc is for pf input; tau_exc for goc is for mf input; tau_exc for mli is for pf input:
# tau_exc = {'golgi': 0.23, 'granule': 5.8, 'purkinje': 1.1, 'basket': 0.64, 'stellate': 0.64, 'dcn': 1.0,
#            'dcnp': 3.64,
#            'io': 1.0}
# tau_inh = {'golgi': 10.0, 'granule': 13.61, 'purkinje': 2.8, 'basket': 2.0, 'stellate': 2.0, 'dcn': 0.7,
#            'dcnp': 1.14, 'io': 60.0}
# tau_exc_cfpc = 0.4
# tau_exc_pfgoc = 0.5
# tau_exc_cfmli = 1.2

# # Single neuron parameters:
# nest_model_builder.neuron_param = {
#         'golgi_cell': {'t_ref': 2.0, 'C_m': 145.0, 'tau_m': 44.0, 'V_th': -55.0, 'V_reset': -75.0, 'Vinit': -62.0,
#                        'E_L': -62.0, 'Vmin': -150.0,
#                        'lambda_0': 1.0, 'tau_V': 0.4, 'I_e': 16.214, 'kadap': 0.217, 'k1': 0.031, 'k2': 0.023,
#                        'A1': 259.988, 'A2': 178.01,
#                        'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc, 'tau_syn1': tau_exc['golgi'],
#                        'tau_syn2': tau_inh['golgi'], 'tau_syn3': tau_exc_pfgoc},
#         'granule_cell': {'t_ref': 1.5, 'C_m': 7.0, 'tau_m': 24.15, 'V_th': -41.0, 'V_reset': -70.0, 'Vinit': -62.0,
#                          'E_L': -62.0, 'Vmin': -150.0,
#                          'lambda_0': 1.0, 'tau_V': 0.3, 'I_e': -0.888, 'kadap': 0.022, 'k1': 0.311, 'k2': 0.041,
#                          'A1': 0.01, 'A2': -0.94,
#                          'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc, 'tau_syn1': tau_exc['granule'],
#                          'tau_syn2': tau_inh['granule'], 'tau_syn3': tau_exc['granule']},
#         'purkinje_cell': {'t_ref': 0.5, 'C_m': 334.0, 'tau_m': 47.0, 'V_th': -43.0, 'V_reset': -69.0, 'Vinit': -59.0,
#                           'E_L': -59.0,
#                           'lambda_0': 4.0, 'tau_V': 3.5, 'I_e': 742.54, 'kadap': 1.492, 'k1': 0.1950, 'k2': 0.041,
#                           'A1': 157.622, 'A2': 172.622,
#                           'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc, 'tau_syn1': tau_exc['purkinje'],
#                           'tau_syn2': tau_inh['purkinje'], 'tau_syn3': tau_exc_cfpc},
#         'basket_cell': {'t_ref': 1.59, 'C_m': 14.6, 'tau_m': 9.125, 'V_th': -53.0, 'V_reset': -78.0, 'Vinit': -68.0,
#                         'E_L': -68.0,
#                         'lambda_0': 1.8, 'tau_V': 1.1, 'I_e': 3.711, 'kadap': 2.025, 'k1': 1.887, 'k2': 1.096,
#                         'A1': 5.953, 'A2': 5.863,
#                         'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc, 'tau_syn1': tau_exc['basket'],
#                         'tau_syn2': tau_inh['basket'], 'tau_syn3': tau_exc_cfmli},
#         'stellate_cell': {'t_ref': 1.59, 'C_m': 14.6, 'tau_m': 9.125, 'V_th': -53.0, 'V_reset': -78.0, 'Vinit': -68.0,
#                           'E_L': -68.0,
#                           'lambda_0': 1.8, 'tau_V': 1.1, 'I_e': 3.711, 'kadap': 2.025, 'k1': 1.887, 'k2': 1.096,
#                           'A1': 5.953, 'A2': 5.863,
#                           'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc, 'tau_syn1': tau_exc['basket'],
#                           'tau_syn2': tau_inh['basket'], 'tau_syn3': tau_exc_cfmli},
#         'dcn_cell': {'t_ref': 0.8, 'C_m': 142.0,'tau_m': 33.0,'V_th': -36.0,'V_reset': -55.0,'Vinit': -45.0,'E_L': -45.0,
#                          'lambda_0':3.5, 'tau_V':3.0,'I_e': 75.385,'kadap': 0.408,'k1': 0.697, 'k2': 0.047,'A1': 13.857,'A2':3.477,
#                          'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc,'tau_syn1': tau_exc['dcn'], 'tau_syn2': tau_inh['dcn']}}

# # Connection weights
# nest_model_builder.conn_weights = {'mossy_to_glomerulus': 1.0, 'ascending_axon_to_golgi': 0.05, 'ascending_axon_to_purkinje': 0.175,
#                     'basket_to_purkinje': 3.638,
#                     'glomerulus_to_golgi': 0.0125, 'glomerulus_to_granule': 0.361, 'golgi_to_granule': 0.338,
#                     'parallel_fiber_to_basket': 0.002, 'parallel_fiber_to_golgi': 0.008,
#                     'parallel_fiber_to_purkinje': 0.044,
#                     'parallel_fiber_to_stellate': 0.003, 'stellate_to_purkinje': 1.213, 
#                     'mossy_to_dcn': 0.5, 'purkinje_to_dcn': 0.45}

# # Connection delays
# nest_model_builder.conn_delays = \
#     {'mossy_to_glomerulus': 1.0, 'ascending_axon_to_golgi': 2.0, 'ascending_axon_to_purkinje': 2.0,
#      'basket_to_purkinje': 4.0, 'glomerulus_to_golgi': 4.0, 'glomerulus_to_granule': 4.0, 'golgi_to_granule': 2.0,
#      'parallel_fiber_to_basket': 5.0, 'parallel_fiber_to_golgi': 5.0, 'parallel_fiber_to_purkinje': 5.0,
#      'parallel_fiber_to_stellate': 5.0, 'stellate_to_purkinje': 5.0, 
#      'mossy_to_dcn': 4.0, 'purkinje_to_dcn': 4.0}

# # Connection receptors
# nest_model_builder.conn_receptors = \
#     {'ascending_axon_to_golgi': 3, 'ascending_axon_to_purkinje': 1, 'basket_to_purkinje': 2,
#      'glomerulus_to_golgi': 1, 'glomerulus_to_granule': 1, 'golgi_to_granule': 2,
#      'parallel_fiber_to_basket': 1, 'parallel_fiber_to_golgi': 3, 'parallel_fiber_to_purkinje': 1,
#      'parallel_fiber_to_stellate': 1, 'stellate_to_purkinje': 2, 
#      'mossy_to_dcn': 1, 'purkinje_to_dcn': 2}

# # Connection pre and post-synaptic neurons
# nest_model_builder.conn_pre_post = \
#     {'mossy_to_glomerulus': {'pre': 'mossy_fibers', 'post': 'glomerulus'},
#      'ascending_axon_to_golgi': {'pre': 'granule_cell', 'post': 'golgi_cell'},
#      'ascending_axon_to_purkinje': {'pre': 'granule_cell', 'post': 'purkinje_cell'},
#      'basket_to_purkinje': {'pre': 'basket_cell', 'post': 'purkinje_cell'},
#      'glomerulus_to_golgi': {'pre': 'glomerulus', 'post': 'golgi_cell'},
#      'glomerulus_to_granule': {'pre': 'glomerulus', 'post': 'granule_cell'},
#      'golgi_to_granule': {'pre': 'golgi_cell', 'post': 'granule_cell'},
#      'parallel_fiber_to_basket': {'pre': 'granule_cell', 'post': 'basket_cell'},
#      'parallel_fiber_to_golgi': {'pre': 'granule_cell', 'post': 'golgi_cell'},
#      'parallel_fiber_to_purkinje': {'pre': 'granule_cell', 'post': 'purkinje_cell'},
#      'parallel_fiber_to_stellate': {'pre': 'granule_cell', 'post': 'stellate_cell'},
#      'stellate_to_purkinje': {'pre': 'stellate_cell', 'post': 'purkinje_cell'}, 
#      'purkinje_to_dcn': {'pre': 'purkinje_cell', 'post': 'dcn_cell'}}

# nest_model_builder.RECORD_VM = True
# nest_model_builder.TOT_DURATION = 350.  # 350 mseconds
# nest_model_builder.STIM_START = 150.  # 150 beginning of stimulation
# nest_model_builder.STIM_END = 250.  # 250 end of stimulation
# nest_model_builder.STIM_FREQ = 100.  # Frequency in Hz
# nest_model_builder.BACKGROUND_FREQ = 4.

# # Load the network source file:
# nest_model_builder.net_src_file = h5py.File(nest_model_builder.path_to_network_source_file, 'r+')

# # Populations' configurations
# nest_model_builder.neuron_types = list(nest_model_builder.net_src_file['cells/placement'].keys())
# ordered_neuron_types = []
# nest_model_builder.ordered_neuron_types = ['mossy_fibers', 'glomerulus', "granule_cell", "golgi_cell",
#                                            "basket_cell", "stellate_cell", "purkinje_cell", "dcn_cell"]
# for neuron_type in nest_model_builder.ordered_neuron_types:
#     ordered_neuron_types.append(
#         nest_model_builder.neuron_types.pop(
#             nest_model_builder.neuron_types.index(neuron_type)))
# ordered_neuron_types += nest_model_builder.neuron_types
# nest_model_builder.neuron_types = ordered_neuron_types

# nest_model_builder.populations = []
# nest_model_builder.start_id_scaffold = {}
# # All cells are modelled as E-GLIF models;
# # with the only exception of Glomeruli and Mossy Fibers (not cells, just modeled as
# # relays; i.e., parrot neurons)
# for neuron_name in nest_model_builder.neuron_types:
#     if neuron_name != 'glomerulus' and neuron_name != 'mossy_fibers':
#         model = 'eglif_cond_alpha_multisyn'
#     else:
#         model = 'parrot_neuron'
#     n_neurons = np.array(nest_model_builder.net_src_file['cells/placement/' + neuron_name + '/identifiers'])[1]
#     nest_model_builder.populations.append(
#                 {"label": neuron_name, "model": model,
#                  "params": nest_model_builder.neuron_param.get(neuron_name, {}),
#                  "scale": n_neurons,
#                  "nodes": None})
#     nest_model_builder.start_id_scaffold[neuron_name] = \
#                 np.array(nest_model_builder.net_src_file['cells/placement/' + neuron_name + '/identifiers'])[0]

# class NeuronsIndsFun(object):
#     conns = np.array([])
#     start_id_scaffold = 0

#     def __init__(self, start_id_scaffold, conns):
#         self.start_id_scaffold = start_id_scaffold
#         self.conns = conns

#     def __call__(self, neurons_inds):
#         return [int(x - self.start_id_scaffold + neurons_inds[0])
#                 for x in self.conns]
    
# # Within brain regions' nodes connections among populations:
# nest_model_builder.default_populations_connection["conn_spec"]["rule"] = "one_to_one"
# nest_model_builder.populations_connections = []
# for conn_name in nest_model_builder.conn_weights.keys():
#     conn = np.array(nest_model_builder.net_src_file['cells/connections/'+conn_name])
#     pre_name = nest_model_builder.conn_pre_post[conn_name]["pre"]
#     post_name = nest_model_builder.conn_pre_post[conn_name]["post"]
#     nest_model_builder.populations_connections.append(
#         {"source": pre_name, "target": post_name,
#          "source_inds": NeuronsIndsFun(nest_model_builder.start_id_scaffold[pre_name], conn[:, 0].flatten()),
#          "target_inds": NeuronsIndsFun(nest_model_builder.start_id_scaffold[post_name], conn[:, 1].flatten()),
#          "synapse_model": 'static_synapse',
#          "conn_spec": nest_model_builder.default_populations_connection["conn_spec"],
#          "weight": nest_model_builder.conn_weights[conn_name],
#          "delay": nest_model_builder.conn_delays[conn_name],
#          "receptor_type": nest_model_builder.conn_receptors.get(conn_name, 0),
#          "nodes": None
#         }
#     )

# # No among brain region regions' nodes' connectivity yet for the NEST Cerebellum network! 
# # Assuming a single Cerebellum region.

# # We don't need the network source file any more. Close it:
# nest_model_builder.net_src_file.close()

# def neurons_inds_fun(neurons_inds, n_neurons=100):
#     # We use this in order to measure up to n_neurons neurons from every population
#     n_neurons_inds = len(neurons_inds)
#     if n_neurons_inds > n_neurons:
#         return tuple(np.array(neurons_inds)[0:-1:int(np.ceil(1.0*n_neurons_inds/n_neurons))])
#     else:
#         return neurons_inds
    
# # Output Devices:
# nest_model_builder.output_devices = []
# # Spike detectors:
# connections = OrderedDict()
# #          label <- target population
# for pop in nest_model_builder.populations:
#     connections[pop["label"] + "_spikes"] = pop["label"]
#     params = dict(nest_model_builder.config.NEST_OUTPUT_DEVICES_PARAMS_DEF["spike_detector"])
#     nest_model_builder.output_devices.append(
#         {"model": "spike_detector", "params": params,
#          "connections": connections, "nodes": None})  # None means all here

# if nest_model_builder.RECORD_VM:    
#     # Multimeters:
#     connections = OrderedDict()
#     #               label    <- target population
#     for pop in nest_model_builder.populations:
#         if pop["label"] != 'glomerulus' and pop["label"] != 'mossy_fibers':
#             connections[pop["label"]] = pop["label"]
#         params = dict(nest_model_builder.config.NEST_OUTPUT_DEVICES_PARAMS_DEF["multimeter"])
#         params["interval"] = nest_model_builder.monitor_period
#         nest_model_builder.output_devices.append(
#                 {"model": "multimeter", "params": params,
#                  "neurons_inds": lambda node, neurons_inds: neurons_inds_fun(neurons_inds),
#                  "connections": connections, "nodes": None})  # None means all here
    
# # Input (stimulus) Devices (needed only when there is no TVB input to the Cerebellum):
# nest_model_builder.input_devices = []
# # # Background spike stimulus :
# # connections = OrderedDict()
# # #             label <- target population
# # connections["Background"] = ['mossy_fibers']
# # nest_model_builder.input_devices.append(
# #     {"model": "poisson_generator",
# #      "params": {"rate": nest_model_builder.BACKGROUND_FREQ, 
# #                 "origin": 0.0, "start": 0.0}, # not necessary: "stop": nest_model_builder.TOT_DURATION
# #      "connections": connections, "nodes": None,
# #      "weights": 1.0, "delays": 0.0, "receptor_type": 0})
# # # Spike stimulus 
# # connections = OrderedDict()
# # #             label <- target population
# # connections["Stimulus"] = ['mossy_fibers']
# # nest_model_builder.input_devices.append(
# #     {"model": "poisson_generator",
# #      "params": {"rate": nest_model_builder.STIM_FREQ, "origin": 0.0, 
# #                 "start": nest_model_builder.STIM_START, "stop": nest_model_builder.STIM_END},
# #      "connections": connections, "nodes": None,
# #      "weights": 1.0, "delays": 0.0, "receptor_type": 0})

# # ----------------------------------------------------------------------------------------------------------------
# # ----------------------------------------------------------------------------------------------------------------
# # ----------------------------------------------------------------------------------------------------------------


# nest_network = nest_model_builder.build_spiking_network()


# or build the network without using the TVB-NEST NEST network builder...:


# ----------------------------------------------------------------------------------------------------------------
# ----Uncomment below to build the NEST network without using the NEST model builder of TVB-NEST------------------
# ----------------------------------------------------------------------------------------------------------------
import h5py
!rm *.gdf
!rm *.dat

nest_model_builder.compile_install_nest_modules(["cereb"])

# Synapse parameters: in E-GLIF, 3 synaptic receptors are present: the first is always associated to exc, the second to inh, the third to remaining synapse type
Erev_exc = 0.0		# [mV]	#[Cavallari et al, 2014]
Erev_inh = -80.0		# [mV]
tau_exc = {'golgi': 0.23, 'granule': 5.8, 'purkinje': 1.1, 'basket': 0.64, 'stellate': 0.64, 'dcn': 1.0, 'dcnp': 3.64, 'io': 1.0}		#tau_exc for pc is for pf input; tau_exc for goc is for mf input; tau_exc for mli is for pf input
tau_inh = {'golgi': 10.0, 'granule': 13.61, 'purkinje': 2.8, 'basket': 2.0, 'stellate': 2.0, 'dcn': 0.7, 'dcnp': 1.14, 'io': 60.0}
tau_exc_cfpc = 0.4
tau_exc_pfgoc = 0.5
tau_exc_cfmli = 1.2

# Single neuron parameters:
neuron_param = {'golgi_cell': {'t_ref': 2.0, 'C_m': 145.0,'tau_m': 44.0,'V_th': -55.0,'V_reset': -75.0,'Vinit': -62.0,'E_L': -62.0,'Vmin':-150.0,
                         'lambda_0':1.0, 'tau_V':0.4,'I_e': 16.214,'kadap': 0.217,'k1': 0.031, 'k2': 0.023,'A1': 259.988,'A2':178.01,
                         'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc,'tau_syn1': tau_exc['golgi'], 'tau_syn2': tau_inh['golgi'], 'tau_syn3': tau_exc_pfgoc},
               'granule_cell': {'t_ref': 1.5, 'C_m': 7.0,'tau_m': 24.15,'V_th': -41.0,'V_reset': -70.0,'Vinit': -62.0,'E_L': -62.0,'Vmin': -150.0,
                           'lambda_0':1.0, 'tau_V':0.3,'I_e': -0.888,'kadap': 0.022,'k1': 0.311, 'k2': 0.041,'A1': 0.01,'A2':-0.94,
                           'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc,'tau_syn1': tau_exc['granule'], 'tau_syn2': tau_inh['granule'], 'tau_syn3': tau_exc['granule']},
               'purkinje_cell': {'t_ref': 0.5, 'C_m': 334.0,'tau_m': 47.0,'V_th': -43.0,'V_reset': -69.0,'Vinit': -59.0,'E_L': -59.0,
                            'lambda_0':4.0, 'tau_V':3.5,'I_e': 742.54,'kadap': 1.492,'k1': 0.1950, 'k2': 0.041,'A1': 157.622,'A2':172.622,
                            'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc,'tau_syn1': tau_exc['purkinje'], 'tau_syn2': tau_inh['purkinje'], 'tau_syn3': tau_exc_cfpc},
               'basket_cell': {'t_ref': 1.59, 'C_m': 14.6,'tau_m': 9.125,'V_th': -53.0,'V_reset': -78.0,'Vinit': -68.0,'E_L': -68.0,
                          'lambda_0':1.8, 'tau_V':1.1,'I_e': 3.711,'kadap': 2.025,'k1': 1.887, 'k2': 1.096,'A1': 5.953,'A2':5.863,
                          'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc,'tau_syn1': tau_exc['basket'], 'tau_syn2': tau_inh['basket'], 'tau_syn3': tau_exc_cfmli},
               'stellate_cell': {'t_ref': 1.59, 'C_m': 14.6,'tau_m': 9.125,'V_th': -53.0,'V_reset': -78.0,'Vinit': -68.0,'E_L': -68.0,
                            'lambda_0':1.8, 'tau_V':1.1,'I_e': 3.711,'kadap': 2.025,'k1': 1.887, 'k2': 1.096,'A1': 5.953,'A2':5.863,
                            'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc,'tau_syn1': tau_exc['basket'], 'tau_syn2': tau_inh['basket'], 'tau_syn3': tau_exc_cfmli},
               'dcn_cell': {'t_ref': 0.8, 'C_m': 142.0,'tau_m': 33.0,'V_th': -36.0,'V_reset': -55.0,'Vinit': -45.0,'E_L': -45.0,
                         'lambda_0':3.5, 'tau_V':3.0,'I_e': 75.385,'kadap': 0.408,'k1': 0.697, 'k2': 0.047,'A1': 13.857,'A2':3.477,
                         'E_rev1': Erev_exc, 'E_rev2': Erev_inh, 'E_rev3': Erev_exc,'tau_syn1': tau_exc['dcn'], 'tau_syn2': tau_inh['dcn']}}


# Connection weights
conn_weights = {'mossy_to_glomerulus': 1.0,'ascending_axon_to_golgi': 0.05, 'ascending_axon_to_purkinje': 0.175, 'basket_to_purkinje': 3.638,\
                'glomerulus_to_golgi': 0.0125, 'glomerulus_to_granule': 0.361, 'golgi_to_granule': 0.338,\
                'parallel_fiber_to_basket': 0.002, 'parallel_fiber_to_golgi': 0.008,'parallel_fiber_to_purkinje': 0.044,\
                'parallel_fiber_to_stellate': 0.003, 'stellate_to_purkinje': 1.213,\
                'mossy_to_dcn': 0.5, 'purkinje_to_dcn': 0.45}


# Connection delays
conn_delays = {'mossy_to_glomerulus': 1.0,'ascending_axon_to_golgi': 2.0, 'ascending_axon_to_purkinje': 2.0, 'basket_to_purkinje': 4.0,
               'glomerulus_to_golgi': 4.0, 'glomerulus_to_granule': 4.0, 'golgi_to_granule': 2.0,
               'parallel_fiber_to_basket': 5.0, 'parallel_fiber_to_golgi': 5.0,'parallel_fiber_to_purkinje': 5.0,
               'parallel_fiber_to_stellate': 5.0, 'stellate_to_purkinje':5.0,\
               'mossy_to_dcn': 4.0, 'purkinje_to_dcn': 4.0}

# Connection receptors
conn_receptors = {'ascending_axon_to_golgi': 3, 'ascending_axon_to_purkinje': 1, 'basket_to_purkinje': 2,
               'glomerulus_to_golgi': 1, 'glomerulus_to_granule': 1, 'golgi_to_granule': 2,
               'parallel_fiber_to_basket': 1, 'parallel_fiber_to_golgi': 3,'parallel_fiber_to_purkinje': 1,
               'parallel_fiber_to_stellate': 1, 'stellate_to_purkinje': 2, \
               'mossy_to_dcn': 1, 'purkinje_to_dcn': 2}


# Connection pre and post-synaptic neurons
conn_pre_post = {'mossy_to_glomerulus': {'pre': 'mossy_fibers', 'post': 'glomerulus'},\
                 'ascending_axon_to_golgi': {'pre': 'granule_cell', 'post': 'golgi_cell'},\
                 'ascending_axon_to_purkinje': {'pre': 'granule_cell', 'post': 'purkinje_cell'},\
                 'basket_to_purkinje': {'pre': 'basket_cell', 'post': 'purkinje_cell'},\
                 'glomerulus_to_golgi': {'pre': 'glomerulus', 'post': 'golgi_cell'}, \
                 'glomerulus_to_granule': {'pre': 'glomerulus', 'post': 'granule_cell'}, \
                 'golgi_to_granule': {'pre': 'golgi_cell', 'post': 'granule_cell'},\
                 'parallel_fiber_to_basket': {'pre': 'granule_cell', 'post': 'basket_cell'}, \
                 'parallel_fiber_to_golgi': {'pre': 'granule_cell', 'post': 'golgi_cell'},\
                 'parallel_fiber_to_purkinje': {'pre': 'granule_cell', 'post': 'purkinje_cell'},\
                 'parallel_fiber_to_stellate': {'pre': 'granule_cell', 'post': 'stellate_cell'}, \
                 'stellate_to_purkinje': {'pre': 'stellate_cell', 'post': 'purkinje_cell'},\
                 'mossy_to_dcn': {'pre': 'mossy_fibers', 'post': 'dcn_cell'},\
                 'purkinje_to_dcn': {'pre': 'purkinje_cell', 'post': 'dcn_cell'}}


# Get the nest instance of the NEST model builder:
nest = nest_model_builder.nest_instance
nest_model_builder._configure_nest_kernel()

# Prepare the NEST kernel:
# nest.ResetKernel()
# nest.set_verbosity('M_ERROR')
# nest.SetKernelStatus({"local_num_threads" : cpu_count()})

# Load the network source file:
f = h5py.File(nest_model_builder.path_to_network_source_file, 'r+')

neuron_types = list(f['cells/placement'].keys())
ordered_neuron_types = []
for neuron_type in ['mossy_fibers', 'glomerulus', "granule_cell", "golgi_cell", 
                    "basket_cell", "stellate_cell", "purkinje_cell"]:
    ordered_neuron_types.append(neuron_types.pop(neuron_types.index(neuron_type)))
ordered_neuron_types += neuron_types
neuron_types = ordered_neuron_types
print(neuron_types)

neuron_number = {}
start_id_scaffold = {}

# ### Creating population of neurons
# Instantiate conductance based Extended-Generalized Leaky Integrate and Fire models (E-GLIF) for each cell type. 
# The only exception is represented by Glomeruli and Mossy Fibers; these are not actual cells but just 'relays', 
# used to deliver input spikes to other cells. Here, Glomeruli are created as *parrot_neurons*

# Create a dictionary; keys = cell names, values = lists to store neuron models
neuron_models = {key: [] for key in neuron_types}

# All cells are modelled as E-GLIF models;
# with the only exception of Glomeruli and Mossy Fibers (not cells, just modeled as
# relays; i.e., parrot neurons)
from tvb_multiscale.tvb_nest.nest_models.brain import NESTBrain
from tvb_multiscale.tvb_nest.nest_models.region_node import NESTRegionNode
from tvb_multiscale.tvb_nest.nest_models.population import NESTPopulation
nest_region_label = simulator.connectivity.region_labels[nest_nodes_ids[0]]
nest_brain = NESTBrain(nest_instance=nest)
nest_brain[nest_region_label] = NESTRegionNode(label=nest_region_label, nest_instance=nest)
for neuron_name in neuron_types:
    if neuron_name  != 'glomerulus' and neuron_name != 'mossy_fibers':
        if neuron_name not in nest.Models():
            model = 'eglif_cond_alpha_multisyn'
            nest.CopyModel('eglif_cond_alpha_multisyn', neuron_name)
            nest.SetDefaults(neuron_name, neuron_param[neuron_name])
    else:
        model = 'parrot_neuron'
        if neuron_name not in nest.Models():
            nest.CopyModel('parrot_neuron', neuron_name)
    
    neuron_number[neuron_name] = np.array(f['cells/placement/'+neuron_name+'/identifiers'])[1]
    start_id_scaffold[neuron_name] = np.array(f['cells/placement/'+neuron_name+'/identifiers'])[0]
    
    neuron_models[neuron_name] = nest.Create(neuron_name, neuron_number[neuron_name])
    
    # Set the neurons' indices into a NESTPopulation class instance inside the NESTBrain class instance:
    nest_brain[nest_region_label][neuron_name] = NESTPopulation(neuron_models[neuron_name],
                                                                label=neuron_name, model=model, 
                                                                nest_instance=nest)

    
# ### Creating synaptic connections
# Here we create synaptic connections among neurons. 
# A message will be printed below the next cell when each connection type is done:

### Load connections from hdf5 file and create them in NEST:
for conn in conn_weights.keys():
    conn_name = conn
    conn = np.array(f['cells/connections/'+conn_name])
    pre_name = conn_pre_post[conn_name]["pre"]
    post_name = conn_pre_post[conn_name]["post"]
    pre = [int(x-start_id_scaffold[pre_name]+neuron_models[pre_name].tolist()[0]) for x in conn[:,0]]
    post = [int(x-start_id_scaffold[post_name]+neuron_models[post_name].tolist()[0]) for x in conn[:,1]]
    if conn_name=="mossy_to_glomerulus":
        syn_param = {"synapse_model": "static_synapse", "weight": conn_weights[conn_name], "delay": conn_delays[conn_name]}
    else:
        syn_param = {"synapse_model": "static_synapse", "weight": conn_weights[conn_name], "delay": conn_delays[conn_name],"receptor_type":conn_receptors[conn_name]}
    nest.Connect(nest.NodeCollection(pre), nest.NodeCollection(post), {"rule": "one_to_one"}, syn_param)
    print("Connections ", conn_name, " done!")

    
from pandas import Series
from tvb_multiscale.core.spiking_models.devices import DeviceSet
from tvb_multiscale.tvb_nest.nest_models.devices import NESTPoissonGenerator, NESTSpikeDetector, NESTMultimeter

input_devices = Series()

# ### Defining stimuli (needed only when there is no TVB input to the Cerebellum)
# Into the next cell, the user can define the parameters value for the simulation. The background input is a 4 Hz Poisson process to glomeruli, for 300 ms. Then a 100-Hz burst is provided, lasting 100 ms. The user can set the following parameters:
# 1. RECORD_VM: by default, spike data are recorded. If you want to record voltage-traces too, please set this variable to 'True', but consider that this is going to increase the computational time of the simulation.
# 2. TOT_DURATION: duration of whole simulation, in milliseconds. 
# 3. STIM_START: when the burst (a Poisson process spike train) should start.
# 4. STIM_END : when the burst should stop.
# 5. STIM_FREQ: frequency of the burst
RECORD_VM = True
TOT_DURATION = 350. # 350 mseconds
STIM_START = 150.   # 150 beginning of stimulation
STIM_END = 250.     # 250 end of stimulation
STIM_FREQ = 100.    # Frequency in Hz 
BACKGROUND_FREQ = 4.

# # Create stimulation devices in NEST and connect to input neural populations (mossy_fibers).
# mossy_fibers_num = len(neuron_models['mossy_fibers'])

# STIM = nest.Create('poisson_generator', params={'rate':STIM_FREQ, 'start': STIM_START, 'stop': STIM_END})
# # STIM as Poisson process
# # TODO: Find out what this is!!!
# # mossy_fibers_pos = np.array(f['cells/placement/mossy_fibers/positions'])
# # x_c, z_c = 200., 200.

# # Connection to glomeruli 
# nest.Connect(STIM, neuron_models['mossy_fibers']) 
# # Set it in a TVB-NEST DeviceSet class instance for the specific brain region:
# input_devices["Stimulus"] = DeviceSet(label="Stimulus", model="poisson_generator")
# input_devices["Stimulus"][nest_region_label] = NESTPoissonGenerator(STIM, nest)

# # Background as Poisson process
# background = nest.Create('poisson_generator',params={'rate':BACKGROUND_FREQ, 'start': 0.0, 'stop': TOT_DURATION}) 
# nest.Connect(background,neuron_models['mossy_fibers'])     
# # Set it in a TVB-NEST DeviceSet class instance for the specific brain region:
# input_devices["Background"] = DeviceSet(label="Background", model="poisson_generator")
# input_devices["Background"][nest_region_label] = NESTPoissonGenerator(background, nest)


# ### Defining recording devices
output_devices = Series()

# Create spike detectors and connect them to the cells; if the user selected RECORD_VM, also voltage will be recorded
## Record spikes from granule and Golgi cells
moss_fib_spikes = nest.Create("spike_detector",
                               params={"record_to": "memory", "label": "mossy_fibers_spikes"})
glom_spikes = nest.Create("spike_detector", params={"record_to": "memory", "label": "glomerulus_spikes"})
grc_spikes = nest.Create("spike_detector", params={"record_to": "memory", "label": "granule_cell_spikes"})
goc_spikes = nest.Create("spike_detector", params={"record_to": "memory", "label": "golgi_cell_spikes"})
bc_spikes = nest.Create("spike_detector", params={"record_to": "memory", "label": "basket_cell_spikes"})
sc_spikes = nest.Create("spike_detector", params={"record_to": "memory", "label": "stellate_cell_spikes"})
pc_spikes = nest.Create("spike_detector", params={"record_to": "memory", "label": "purkinje_cell_spikes"})
dcn_spikes = nest.Create("spike_detector", params={"record_to": "memory", "label": "dcn_cell_spikes"})

# Here you can choose which devices you want to connect and thus the neural populations you want to record.
# Increasing the number of recorded cells can increase the duration of the simulation
nest.Connect(neuron_models['mossy_fibers'], moss_fib_spikes)
nest.Connect(neuron_models['glomerulus'], glom_spikes)
nest.Connect(neuron_models['granule_cell'], grc_spikes)
nest.Connect(neuron_models['golgi_cell'], goc_spikes)
nest.Connect(neuron_models['basket_cell'], bc_spikes)
nest.Connect(neuron_models['stellate_cell'], sc_spikes)
nest.Connect(neuron_models['purkinje_cell'], pc_spikes)
nest.Connect(neuron_models['dcn_cell'], dcn_spikes)

# Set them in a TVB-NEST DeviceSet class instance for the specific brain region:
for device_id, label in zip([moss_fib_spikes, glom_spikes, grc_spikes, goc_spikes, 
                             bc_spikes, sc_spikes, pc_spikes, dcn_spikes], 
                            ['mossy_fibers', "glomerulus_spikes", "granule_cell_spikes", "golgi_cell_spikes", 
                             "basket_cell_spikes", "stellate_cell_spikes", "purkinje_cell_spikes", "dcn_cell_spikes"]):
    output_devices[label] = DeviceSet(label=label, model="spike_detector")
    output_devices[label][nest_region_label] = NESTSpikeDetector(device_id, nest)

    
if RECORD_VM:
    
    def neurons_inds_fun(neurons_inds, n_neurons=100):
        # We use this in order to measure up to n_neurons neurons from every population
        n_neurons_inds = len(neurons_inds)
        if n_neurons_inds > n_neurons:
            return tuple(np.array(neurons_inds)[0:-1:int(np.ceil(1.0*n_neurons_inds/n_neurons))])
        else:
            return neurons_inds
            
    print("Recording membrane voltage")
    grc_vm = nest.Create("multimeter")
    goc_vm = nest.Create("multimeter")
    bc_vm = nest.Create("multimeter")
    sc_vm = nest.Create("multimeter")                     
    pc_vm = nest.Create("multimeter")
    dcn_vm = nest.Create("multimeter")

    nest.SetStatus(grc_vm, {"record_from": ["V_m"], "record_to": "memory", "label": "granule_cell_vm"})
    nest.SetStatus(goc_vm, {"record_from": ["V_m"], "record_to": "memory", "label": "golgi_cell_vm"})
    nest.SetStatus(bc_vm, {"record_from": ["V_m"], "record_to": "memory", "label": "basket_cell_vm"})
    nest.SetStatus(sc_vm, {"record_from": ["V_m"], "record_to": "memory", "label": "stellate_cell_vm"})
    nest.SetStatus(pc_vm, {"record_from": ["V_m"], "record_to": "memory", "label": "purkinje_cell_vm"})
    nest.SetStatus(dcn_vm, {"record_from": ["V_m"], "record_to": "memory", "label": "dcn_cell_vm"})

    nest.Connect(grc_vm, neurons_inds_fun(neuron_models['granule_cell']))
    nest.Connect(goc_vm, neurons_inds_fun(neuron_models['golgi_cell']))
    nest.Connect(bc_vm, neurons_inds_fun(neuron_models['basket_cell']))
    nest.Connect(sc_vm, neurons_inds_fun(neuron_models['stellate_cell']))
    nest.Connect(pc_vm, neurons_inds_fun(neuron_models['purkinje_cell']))
    nest.Connect(dcn_vm, neurons_inds_fun(neuron_models['dcn_cell']))

# Set them in a TVB-NEST DeviceSet class instance for the specific brain region:
for device_id, label in zip([grc_vm, goc_vm, bc_vm, 
                             sc_vm, pc_vm, dcn_vm], 
                            ["granule_cell_vm", "golgi_cell_vm", "basket_cell_vm", 
                             "stellate_cell_vm", "purkinje_cell_vm", "dcn_cell_vm"]):
    output_devices[label] = DeviceSet(label=label, model="multimeter")
    output_devices[label][nest_region_label] = NESTMultimeter(device_id, nest)

    
f.close()

# Finally construct the NEST network model:
from tvb_multiscale.tvb_nest.nest_models.network import NESTNetwork
nest_model_builder._update_default_min_delay()
nest_network = NESTNetwork(nest_instance=nest,
                           brain_regions=nest_brain,
                           output_devices=output_devices,
                           input_devices=input_devices, 
                           config=nest_model_builder.config)

# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------




In [ ]:
if "nest_model_builder" in globals():
    populations_sizes = []
    print("Population sizes: ")
    for pop in nest_model_builder.populations:
        populations_sizes.append(int(np.round(pop["scale"] * nest_model_builder.population_order)))
        print("%s: %d" % (pop["label"], populations_sizes[-1]))
if "nest_network" in globals():
    print(nest_network.print_str(connectivity=True))
else:
    nest_network = None

## 3. Build the TVB-NEST interface

In [ ]:
from tvb_multiscale.tvb_nest.interfaces.builders.models.linear_cereb import LinearCerebBuilder

# Build a TVB-NEST interface with all the appropriate connections between the
# TVB and NEST modelled regions
tvb_nest_builder = \
    LinearCerebBuilder(simulator, nest_network, nest_nodes_ids, 
                       exclusive_nodes=True, populations_sizes=populations_sizes)

tvb_to_nest_mode = "rate"
nest_to_tvb = True

# Using all default parameters for this example

# or...


# ----------------------------------------------------------------------------------------------------------------
# ----Uncomment below to modify the builder by changing the default options:--------------------------------------
# ----------------------------------------------------------------------------------------------------------------


# TVB -> NEST


# --------For spike transmission from TVB to NEST devices acting as TVB proxy nodes with TVB delays:--------
from tvb_multiscale.core.spiking_models.builders.templates import scale_tvb_weight, tvb_delay

tvb_nest_builder.G = tvb_nest_builder.tvb_simulator.model.G[0].item()
tvb_nest_builder.global_coupling_scaling = tvb_nest_builder.tvb_simulator.coupling.a[0].item() * tvb_nest_builder.G
    
tvb_weight_fun = \
    lambda tvb_node_id: 200*tvb_nest_builder.global_coupling_scaling * \
                        tvb_nest_builder.tvb_weights[tvb_node_id, tvb_nest_builder.spiking_nodes_ids].sum()           

tvb_delay_fun = \
    lambda source_node, target_node: \
        np.maximum(tvb_nest_builder.tvb_dt, tvb_delay(source_node, target_node, tvb_nest_builder.tvb_delays))

tvb_nest_builder.tvb_to_spikeNet_interfaces = []
if tvb_to_nest_mode == "rate":
    # Mean spike rates are applied in parallel to all target neurons

    tvb_nest_builder.tvb_to_spikeNet_interfaces.append(
            {"model": "inhomogeneous_poisson_generator",
             "params": {"allow_offgrid_times": False},
             # ---Property potentially set as function handles with args (tvb_node_id=None),-----------
             # ---applied outside NEST for each interface device-------------------------------------
             "interface_weights": tvb_weight_fun, 
             # ----Properties potentially set as function handles with args (tvb_node_id=None, nest_node_id=None)---
            "weights": 1.0, # lambda tvb_node_id, nest_node_id: tvb_weight_fun(tvb_node_id, nest_node_id),
            "delays": tvb_delay_fun,
            "receptor_type": 0,
             # -----------------------------------------------------------------------------------------------------
             #            TVB sv -> NEST population
            "connections": {"R": "mossy_fibers"},
            "source_nodes": None, "target_nodes": None} # None means all nodes
    )   
    
if nest_to_tvb:
    tvb_nest_builder.spikeNet_to_tvb_interfaces = []
    # TVB <-- NEST:
    connections = OrderedDict()    
    #            TVB <- NEST
    connections["Rin"] = "dcn_cell"
    tvb_nest_builder.spikeNet_to_tvb_interfaces.append(
            {"model": "spike_detector", "params": {},
             # --Properties potentially set as function handles with args (nest_node_id=None)----
             "interface_weights": 5.0, "delays": 0.0,
             # ----------------------------------------------------------------------------------
             "connections": {"Rin": "dcn_cell"}, "nodes": None})  # None means all nodes

tvb_nest_builder.w_tvb_to_spike_rate = 1.0
# We return from a NEST spike_detector the ratio number_of_population_spikes / number_of_population_neurons
# for every TVB time step, which is already a quantity in the range [0.0, 1.0],
# as long as a neuron cannot fire twice during a TVB time step, i.e.,
# as long as the TVB time step (usually 0.001 to 0.1 ms)
# is smaller than the neurons' refractory time, t_ref (usually 1-2 ms)
tvb_nest_builder.w_spikes_to_tvb = 1000.0

# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------

tvb_nest_model = tvb_nest_builder.build_interface(tvb_to_nest_mode=tvb_to_nest_mode, nest_to_tvb=nest_to_tvb)




In [ ]:
if "tvb_nest_model" in globals():
    print(tvb_nest_model.print_str(detailed_output=True, connectivity=False))
else:
    tvb_nest_model = None

## 4. Configure simulator, simulate, gather results

In [ ]:
# Set the simulation and transient (to be optionally removed from resutls) times:
simulation_length = nest_model_builder.TOT_DURATION
transient = nest_model_builder.STIM_START / 3
if transient:
    exclude_times = [0.0, transient]
else:
    exclude_times = []

In [ ]:
# Configure the simulator with the TVB-NEST interface...
# ...and simulate!
tic = time.time()
if tvb_nest_model is None:
    print("Simulating only NEST...")
    nest_network.nest_instance.Prepare()
    # Integrate NEST for simulation_length + 1 NEST time step so that multimeters get the last time point
    # unless you plan to continue simulation later
    nest_network.nest_instance.Run(simulation_length + nest_network.nest_instance.GetKernelStatus("resolution"))
    nest_network.nest_instance.Cleanup()
    results = None
else:
    print("Simulating TVB-NEST...")
    simulator.configure(tvb_spikeNet_interface=tvb_nest_model)
    # results = simulator.run(simulation_length=simulation_length)
    print("...simulating brackground resting state...")
    results1 = simulator.run(simulation_length=nest_model_builder.STIM_START)
    print("...simulating stimulus activity...")
    simulator.model.I_o[stim_node_id] = 10.0 # 0.75
    results2 = simulator.run(simulation_length=nest_model_builder.STIM_END-nest_model_builder.STIM_START, 
                             configure_spiking_simulator=False)
    print("...simulating relaxation to resting state...")
    simulator.model.I_o[stim_node_id] = 0.0
    results3 = simulator.run(simulation_length=simulation_length-nest_model_builder.STIM_END, 
                             configure_spiking_simulator=False)
    results = [[np.concatenate([results1[0][0], results2[0][0], results3[0][0]]),  # concat time
                np.concatenate([results1[0][1], results2[0][1], results3[0][1]])]] # concat data
    del results1, results2, results3
    # Integrate NEST for one more NEST time step so that multimeters get the last time point
    # unless you plan to continue simulation later
    simulator.run_spiking_simulator(simulator.tvb_spikeNet_interface.nest_instance.GetKernelStatus("resolution"))
    # Clean-up NEST simulation
    simulator.tvb_spikeNet_interface.nest_instance.Cleanup()
print("\nSimulated in %f secs!" % (time.time() - tic))

## 5. Plot results and write them to HDF5 files

In [ ]:
# set to False for faster plotting of only mean field variables and dates, apart from spikes" rasters:
plot_per_neuron = True  
# from examples.plot_write_results import plot_write_results
# populations = []
# populations_sizes = []
# for pop in nest_model_builder.populations:
#     populations.append(pop["label"])
#     populations_sizes.append(int(np.round(pop["scale"] * nest_model_builder.population_order)))
# plot_write_results(results, simulator, populations=populations, populations_sizes=populations_sizes, 
#                    transient=transient, tvb_state_variable_type_label="State Variables", 
#                    tvb_state_variables_labels=simulator.model.variables_of_interest, 
#                    plot_per_neuron=plot_per_neuron, plotter=plotter, config=config)

In [ ]:
# If you want to see what the function above does, take the steps, one by one
try:
    # We need framework_tvb for writing and reading from HDF5 files
    from tvb_multiscale.core.io.h5_writer import H5Writer
    writer = H5Writer()
except:
    writer = False
    
from tvb.contrib.scripts.datatypes.time_series import TimeSeriesRegion
from tvb.contrib.scripts.datatypes.time_series_xarray import TimeSeriesRegion as TimeSeriesXarray

# Put the results in a Timeseries instance
from tvb.contrib.scripts.datatypes.time_series import TimeSeriesRegion

### 5a. TVB results


In [ ]:
source_ts = None
if results is not None:
    source_ts = TimeSeriesXarray(  # substitute with TimeSeriesRegion fot TVB like functionality
            data=results[0][1], time=results[0][0],
            connectivity=simulator.connectivity,
            labels_ordering=["Time", "State Variable", "Region", "Neurons"],
            labels_dimensions={"State Variable": list(simulator.model.variables_of_interest),
                               "Region": simulator.connectivity.region_labels.tolist()},
            sample_period=simulator.integrator.dt)
    source_ts.configure()

    # Remove transient, if any
    if transient:
        source_ts = source_ts[transient:]
        exclude_times = [0.0, transient]
    else:
        exclude_times = []
    t = source_ts.time

    # Write to file
    if writer:
        writer.write_tvb_to_h5(TimeSeriesRegion().from_xarray_DataArray(source_ts._data,
                                                                        connectivity=source_ts.connectivity),
                               os.path.join(config.out.FOLDER_RES, source_ts.title)+".h5")
    source_ts
    

In [ ]:
if source_ts is not None:
    # Plot TVB time series
    source_ts.plot_timeseries(plotter_config=plotter.config, per_variable=True, figsize=FIGSIZE, add_legend=False);

In [ ]:
if source_ts is not None:
    # TVB time series raster plot:
    if source_ts.number_of_labels > 9:
        source_ts.plot_raster(plotter_config=plotter.config, per_variable=True, figsize=FIGSIZE, add_legend=False);

In [ ]:
# A more informative figure with the 10 regions with highest mean firing rate and legend:
MeanRate = source_ts.data[:, 0].mean(axis=0).flatten()
inds = np.argsort(MeanRate)[::-1][:10]
source_ts[:,"R",inds, :].plot_timeseries(plotter_config=plotter.config, per_variable=True, figsize=FIGSIZE, add_legend=True);
from matplotlib import pyplot
fontsize = 20
for ax in pyplot.gcf().axes:
#     ax.get_yticks().set_fontsize(fontsize)
    ax.set_xticks([])
    ax.set_xlabel("")
    ax.set_title(ax.get_ylabel(), fontsize=fontsize)
    ax.set_ylabel("")
    for text in ax.get_legend().get_texts():
        text.set_fontsize(16)
    ax.get_legend().get_title().set_fontsize(18)
    
source_ts[:,"Rin",inds, :].plot_timeseries(plotter_config=plotter.config, per_variable=True, figsize=FIGSIZE, add_legend=False);
for ax in pyplot.gcf().axes:
#     ax.get_yticks().set_fontsize(fontsize)
#     ax.get_xticks().set_fontsize(fontsize)
    ax.set_xlabel(ax.get_xlabel(), fontsize=fontsize)
    ax.set_title(ax.get_ylabel(), fontsize=fontsize)
    ax.set_ylabel("")
    

### Interactive time series plot

In [ ]:
# # ...interactively as well
# # For interactive plotting:
# %matplotlib notebook 
# plotter.plot_timeseries_interactive(source_ts)

In [ ]:
# The stimunlated region:
# source_ts[:, :, stim_node_id].plot_timeseries(plotter_config=plotter.config, per_variable=True, figsize=FIGSIZE,
#                                              figname="Stimulated region TVB Time Series");

In [ ]:
# # Focus on the nodes modelled in NEST:
# try:
#     tvb_nest_model = simulator.tvb_spikeNet_interface
# except:
#     pass
# source_ts_nest = None
# if tvb_nest_model is not None:
#     n_spiking_nodes = len(simulator.tvb_spikeNet_interface.spiking_nodes_ids)
#     source_ts_nest = source_ts[:, :, simulator.tvb_spikeNet_interface.spiking_nodes_ids]
#     source_ts_nest.plot_timeseries(plotter_config=plotter.config, per_variable=True, figsize=FIGSIZE,
#                                    figname="Spiking nodes TVB Time Series");


In [ ]:
# # Focus on the nodes modelled in NEST: raster plot
# if source_ts_nest is not None:
#     source_ts_nest.plot_raster(plotter_config=plotter.config, per_variable=True, figsize=FIGSIZE,
#                                figname="Spiking nodes TVB Time Series Raster");

### 5b. Spiking Network plots

In [ ]:
from tvb_multiscale.tvb_elephant.spiking_network_analyser import SpikingNetworkAnalyser
# Create a SpikingNetworkAnalyzer:
spikeNet_analyzer = \
    SpikingNetworkAnalyser(spikeNet=nest_network,
                           start_time=source_ts.time[0], end_time=source_ts.time[-1], 
                           period=simulator.monitors[0].period,
                           time_series_output_type="TVB", return_data=True, 
                           force_homogeneous_results=True, connectivity=simulator.connectivity)

### Plot spikes' raster and mean spike rates and correlations

In [ ]:
# Spikes rates and correlations per Population and Region
spikes_res = \
    spikeNet_analyzer.\
        compute_spikeNet_spikes_rates_and_correlations(
            populations_devices=None, regions=None,
            rates_methods=[], rates_kwargs=[{}],rate_results_names=[],
            corrs_methods=[], corrs_kwargs=[{}], corrs_results_names=[], bin_kwargs={},
            data_method=spikeNet_analyzer.get_spikes_from_device, data_kwargs={},
            return_devices=False
        );

In [ ]:
if spikes_res:
    print(spikes_res["mean_rate"])
    print(spikes_res["spikes_correlation_coefficient"])
    # Plot spikes' rasters together with mean population's spikes' rates' time series
    if plotter:
        plotter.plot_spike_events(spikes_res["spikes"], rates=spikes_res["mean_rate_time_series"], figsize=FIGSIZE)
        from tvb_multiscale.core.plot.correlations_plot import plot_correlations
        plot_correlations(spikes_res["spikes_correlation_coefficient"], plotter)

In [ ]:
if spikes_res and writer:
    writer.write_object(spikes_res["spikes"].to_dict(), 
                        path=os.path.join(config.out.FOLDER_RES,  "Spikes") + ".h5");
    writer.write_object(spikes_res["mean_rate"].to_dict(),
                        path=os.path.join(config.out.FOLDER_RES,
                                          spikes_res["mean_rate"].name) + ".h5");
    writer.write_tvb_to_h5(TimeSeriesRegion().from_xarray_DataArray(
                              spikes_res["mean_rate_time_series"]._data,
                               connectivity=spikes_res["mean_rate_time_series"].connectivity),
                           os.path.join(config.out.FOLDER_RES,
                                        spikes_res["mean_rate_time_series"].title) + ".h5",
                           recursive=False);
    writer.write_object(spikes_res["spikes_correlation_coefficient"].to_dict(),
                        path=os.path.join(config.out.FOLDER_RES,
                                          spikes_res["spikes_correlation_coefficient"].name) + ".h5");

### Get  SpikingNetwork mean field variable time series and plot them

In [ ]:
# Continuous time variables' data of spiking neurons
if plot_per_neuron:
    spikeNet_analyzer.return_data = True
else:
    spikeNet_analyzer.return_data = False
spikeNet_ts = \
    spikeNet_analyzer. \
         compute_spikeNet_mean_field_time_series(populations_devices=None, regions=None, variables=None,
                                                 computations_kwargs={}, data_kwargs={}, return_devices=False)
if spikeNet_ts:
    if plot_per_neuron:
        mean_field_ts = spikeNet_ts["mean_field_time_series"]  # mean field
        spikeNet_ts = spikeNet_ts["data_by_neuron"]  # per neuron data
    else:
        mean_field_ts = spikeNet_ts
        spikeNet_ts = None
    if mean_field_ts and mean_field_ts.size > 0:
        mean_field_ts.plot_timeseries(plotter_config=plotter.config, per_variable=True)
        if mean_field_ts.shape[2] > 3:
            mean_field_ts.plot_raster(plotter_config=plotter.config, per_variable=True,
                                      linestyle="--", alpha=0.5, linewidth=0.5)
else:
    mean_field_ts = None

In [ ]:
# Write results to file:
if mean_field_ts and writer:
    writer.write_tvb_to_h5(TimeSeriesRegion().from_xarray_DataArray(
                                       mean_field_ts._data,
                                       connectivity=mean_field_ts.connectivity),
                           os.path.join(config.out.FOLDER_RES, mean_field_ts.title) + ".h5", 
                           recursive=False)

### Compute per neuron spikes' rates times series and plot them

In [ ]:
if spikes_res and plot_per_neuron:
    from tvb.simulator.plot.base_plotter import pyplot
    spikeNet_analyzer.return_data = False
    rates_ts_per_neuron = \
        spikeNet_analyzer. \
            compute_spikeNet_rates_time_series(populations_devices=None, regions=None,
                                               computations_kwargs={}, data_kwargs={},
                                               return_spikes_trains=False, return_devices=False);
    if rates_ts_per_neuron is not None and rates_ts_per_neuron.size:
        # Regions in rows
        row = rates_ts_per_neuron.dims[2] if rates_ts_per_neuron.shape[2] > 1 else None
        if row is None:
            # Populations in rows
            row = rates_ts_per_neuron.dims[1] if rates_ts_per_neuron.shape[1] > 1 else None
            col = None
        else:
            # Populations in columns
            col = rates_ts_per_neuron.dims[1] if rates_ts_per_neuron.shape[1] > 1 else None
        pyplot.figure()
        rates_ts_per_neuron.plot(y=rates_ts_per_neuron.dims[3], row=row, col=col, cmap="jet")
        plotter.base._save_figure(figure_name="Spike rates per neuron")
        # del rates_ts_per_neuron # to free memory

### Plot per neuron SpikingNetwork time series

In [ ]:
# Regions in rows
if spikeNet_ts is not None and spikeNet_ts.size:
    row = spikeNet_ts.dims[2] if spikeNet_ts.shape[2] > 1 else None
    if row is None:
        # Populations in rows
        row = spikeNet_ts.dims[3] if spikeNet_ts.shape[3] > 1 else None
        col = None
    else:
        # Populations in cols
         col = spikeNet_ts.dims[3] if spikeNet_ts.shape[3] > 1 else None
    for var in spikeNet_ts.coords[spikeNet_ts.dims[1]]:
        this_var_ts = spikeNet_ts.loc[:, var, :, :, :]
        this_var_ts.name = var.item()
        pyplot.figure()
        this_var_ts.plot(y=spikeNet_ts.dims[4], row=row, col=col, cmap="jet", figsize=FIGSIZE)
        plotter.base._save_figure(
            figure_name="Spiking Network variables' time series per neuron: %s" % this_var_ts.name)
    del spikeNet_ts # to free memory

# References

1 Paula Sanz Leon, Stuart A. Knock, M. Marmaduke Woodman, Lia Domide, <br>
  Jochen Mersmann, Anthony R. McIntosh, Viktor Jirsa (2013) <br>
  The Virtual Brain: a simulator of primate brain network dynamics. <br>
  Frontiers in Neuroinformatics (7:10. doi: 10.3389/fninf.2013.00010) <br>
  https://www.thevirtualbrain.org/tvb/zwei <br>
  https://github.com/the-virtual-brain <br>

2 Ritter P, Schirner M, McIntosh AR, Jirsa VK. 2013.  <br>
  The Virtual Brain integrates computational modeling  <br>
  and multimodal neuroimaging. Brain Connectivity 3:121–145. <br>

3 Jordan, Jakob; Mørk, Håkon; Vennemo, Stine Brekke;   Terhorst, Dennis; Peyser, <br>
  Alexander; Ippen, Tammo; Deepu, Rajalekshmi;   Eppler, Jochen Martin; <br>
  van Meegen, Alexander;   Kunkel, Susanne; Sinha, Ankur; Fardet, Tanguy; Diaz, <br>
  Sandra; Morrison, Abigail; Schenck, Wolfram; Dahmen, David;   Pronold, Jari; <br>
  Stapmanns, Jonas;   Trensch, Guido; Spreizer, Sebastian;   Mitchell, Jessica; <br>
  Graber, Steffen; Senk, Johanna; Linssen, Charl; Hahne, Jan; Serenko, Alexey; <br>
  Naoumenko, Daniel; Thomson, Eric;   Kitayama, Itaru; Berns, Sebastian;   <br>
  Plesser, Hans Ekkehard <br>
  NEST is a simulator for spiking neural network models that focuses <br>
  on the dynamics, size and structure of neural systems rather than on <br>
  the exact morphology of individual neurons. <br>
  For further information, visit http://www.nest-simulator.org. <br>
  The release notes for this release are available at  <br>
  https://github.com/nest/nest-simulator/releases/tag/v2.18.0 <br>